### Install the libraries

In [ ]:
! pip install llama_index


In [ ]:
! pip install llama-index-llms-huggingface


In [ ]:
! pip install llama-index-llms-huggingface-api


In [ ]:
! pip install llama-index-embeddings-huggingface


In [ ]:
! pip install vllm

In [ ]:
!pip install -U llama-index llama-index-llms-groq groq

In [ ]:
!pip uninstall -y pinecone-client

!pip install pinecone

In [ ]:
!pip install llama-index-core llama-index-vector-stores-pinecone


In [ ]:
!pip install --no-cache-dir pinecone

### Import the libraries

In [4]:
from llama_index.core import SimpleDirectoryReader, Settings, StorageContext, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
import pinecone
from pinecone import ServerlessSpec

### Setting up the LLM => gives context

In [5]:
groq_api_key = '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'

llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    temperature=0
)

Settings.llm=llm

### Setting up the embedder model => convert text to numerical representation

In [ ]:

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)
Settings.embed_model = embed_model

### Initialise Pinecone

In [7]:
pc = pinecone.Pinecone(api_key="pcsk_2q5Ago_2d7VomYRS1qaNm67EA3ufv4aHgfzLAogwnG4hzBE2jXZwphtZq4nk9YRZSmJNvp")

### Indexing

In [8]:
index_name = "llamaindex-demo"

# Check if the index already exists, otherwise create one
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to the index just created
pinecone_index = pc.Index(index_name)

### Create the Pinecone vector store for LlamaIndex

In [9]:
# Persist the index to a storage folder
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)


### Create a storage context



In [10]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

### Streamlit app

In [ ]:
!pip install streamlit pyngrok PyPDF2

In [ ]:
from pyngrok import ngrok
import streamlit as st
import os, shutil

ngrok.set_auth_token("")


In [ ]:
%%writefile app.py
import streamlit as st
import os, shutil
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.pinecone import PineconeVectorStore
import pinecone
from pinecone import ServerlessSpec
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

GROQ_API_KEY = ""
PINECONE_API_KEY = ""
INDEX_NAME = "llamaindex-demo"

st.set_page_config(page_title="AI-Powered Document Search & Summarization", page_icon="📄", layout="centered")
st.title("📄 AI-Powered Document Search & Summarization")

uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])
question = st.text_input("Ask a question about your document:")
submit = st.button("Ask")

if uploaded_file and submit:
    try:
        st.info("🚀 Starting process...")
        print("🚀 Starting process...")

        # Step 1: Save uploaded file
        st.write("🔹 Step 1: Saving uploaded PDF...")
        print("Saving uploaded PDF...")
        if os.path.exists("papers"):
            shutil.rmtree("papers")
        os.makedirs("papers", exist_ok=True)
        file_path = os.path.join("papers", uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        st.success(f"✅ File saved: {uploaded_file.name}")
        print(f"File saved at {file_path}")

        # Step 2: Load document
        st.write("🔹 Step 2: Loading document with LlamaIndex...")
        print("Loading document with SimpleDirectoryReader...")
        docs = SimpleDirectoryReader("./papers").load_data()
        st.success(f"✅ Document loaded. Found {len(docs)} chunks.")
        print(f"Document loaded with {len(docs)} chunks.")

        # Step 3: Initialize LLM and embedding model
        st.write("🔹 Step 3: Initializing models...")
        print("Initializing Groq LLM and embedding model...")
        llm = Groq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
        Settings.llm = llm
        Settings.embed_model = embed_model
        st.success("✅ Models initialized.")
        print("Models initialized successfully.")

        # Step 4: Connect to Pinecone
        st.write("🔹 Step 4: Connecting to Pinecone...")
        print("Connecting to Pinecone...")
        pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
        if INDEX_NAME not in [i.name for i in pc.list_indexes()]:
            st.write("Creating new Pinecone index...")
            pc.create_index(
                name=INDEX_NAME,
                dimension=1024,
                metric="cosine",
                spec=ServerlessSpec(cloud='aws', region='us-east-1')
            )
            st.success("✅ New Pinecone index created.")
        pinecone_index = pc.Index(INDEX_NAME)
        vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        st.success("✅ Connected to Pinecone successfully.")
        print("Connected to Pinecone successfully.")

        # Step 5: Build index
        st.write("🔹 Step 5: Building index (this may take a few seconds)...")
        print("Building index from documents...")
        index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, show_progress=True)
        query_engine = index.as_query_engine(similarity_top_k=5, response_mode="tree_summarize")
        st.success("✅ Index built successfully.")
        print("Index built successfully.")

        # Step 6: Query
        st.write("🔹 Step 6: Running query...")
        print(f"Running query: {question}")
        response = query_engine.query(question)
        st.subheader("✅ Answer:")
        st.write(str(response))
        print("Query completed successfully.")

    except Exception as e:
        st.error(f"❌ Error: {str(e)}")
        print(f"❌ Error occurred: {str(e)}")

elif submit and not uploaded_file:
    st.warning("⚠️ Please upload a PDF file first.")


Writing app.py


In [ ]:
!streamlit run app.py --server.port 8501 &> /content/logs.txt &
ngrok_tunnel = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print(' * Tunnel URL:', ngrok_tunnel.public_url)

In [ ]:
!tail -f /content/logs.txt
